In [ ]:
# default_exp clone_counters

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# export
from functools import partial
from glob import glob

import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
import xarray as xr
from skimage import measure

from py_clone_detective.utils import (
    add_scale_regionprops_table_area_measurements,
    calculate_overlap,
    check_channels_input_suitable_and_return_channels,
    extend_region_properties_list,
    get_all_labeled_clones_unmerged_and_merged,
    img_path_to_xarr,
    last2dims,
    lazy_props,
    reorder_df_to_put_ch_info_first,
    update_1st_coord_and_dim_of_xarr,
)

# CloneCounter Classes

## Parent Class

In [ ]:
# export
class CloneCounter:
    def __init__(self, exp_name: str, img_name_regex: str, pixel_size: float):
        self.exp_name, self.img_name_regex, self.pixel_size = (
            exp_name,
            img_name_regex,
            pixel_size,
        )

    def add_images(self, **channel_path_globs):
        return img_path_to_xarr(
            self.img_name_regex,
            self.pixel_size,
            ch_name_for_first_dim="img_channels",
            **channel_path_globs,
        )

    def add_segmentations(self, **channel_path_globs):
        segmentations = img_path_to_xarr(
            self.img_name_regex,
            self.pixel_size,
            ch_name_for_first_dim="seg_channels",
            **channel_path_globs,
        )
        segmentations.data = segmentations.data.map_blocks(
            last2dims(partial(measure.label)), dtype=np.uint16
        )
        return segmentations

    def combine_C0_overlaps_and_measurements(self):
        ov_df = (
            self.results_overlaps.pivot(
                index=["img_name", "C0_labels"],
                columns=["colocalisation_ch"],
                values="is_in_label",
            )
            .query("C0_labels != 0")
            .copy()
        )
        sk_df = self.results_measurements.query("seg_channel == 'C0'").set_index(
            ["segmentation_img_name", "label"]
        )
        sk_df.index.rename(["img_name", "C0_labels"], inplace=True)
        return pd.merge(ov_df, sk_df, left_index=True, right_index=True)

    def determine_seg_img_channel_pairs(
        self, seg_channels: list = None, img_channels: list = None
    ):
        seg_channels = check_channels_input_suitable_and_return_channels(
            channels=seg_channels,
            available_channels=self.image_data.seg_channels.values.tolist(),
        )

        img_channels = check_channels_input_suitable_and_return_channels(
            channels=img_channels,
            available_channels=self.image_data.img_channels.values.tolist(),
        )

        seg_img_channel_pairs = pd.DataFrame()
        seg_img_channel_pairs["image_channel"] = pd.Series(img_channels)
        seg_img_channel_pairs["segmentation_channel"] = pd.Series(seg_channels)
        self.seg_img_channel_pairs = seg_img_channel_pairs.fillna(method="ffill")[
            ["segmentation_channel", "image_channel"]
        ]

    def make_measurements(
        self,
        seg_channels: list = None,
        img_channels: list = None,
        extra_properties: list = None,
        **kwargs,
    ):

        self.determine_seg_img_channel_pairs(seg_channels, img_channels)

        properties = extend_region_properties_list(extra_properties)

        results = list()
        for _, seg_ch, img_ch in self.seg_img_channel_pairs.itertuples():
            for seg, img in zip(
                self.image_data["segmentations"].loc[seg_ch],
                self.image_data["images"].loc[img_ch],
            ):
                results.append(
                    lazy_props(
                        seg.data,
                        img.data,
                        seg.seg_channels.item(),
                        img.img_channels.item(),
                        seg.img_name.item(),
                        img.img_name.item(),
                        properties,
                        **kwargs,
                    )
                )

        df = dd.from_delayed(results).compute()
        df = add_scale_regionprops_table_area_measurements(df, self.pixel_size)
        self.results_measurements = reorder_df_to_put_ch_info_first(df)

    def _determine_max_seg_label_levels(self, seg_channel):
        """Determines in self.max_seg_label_levels is unassigned and create empty.
        Then adds `seg_channel` as new key and corresponding max segmentation label levels
        as value"""

        try:
            self.max_seg_label_levels.setdefault(seg_channel, []).append(
                (
                    self.image_data["segmentations"]
                    .loc[seg_channel]
                    .data.map_blocks(
                        lambda x: np.unique(x).shape[0],
                        drop_axis=(1, 2),
                        dtype=np.uint16,
                    )
                    .compute()
                    .max()
                )
            )
            self.max_seg_label_levels[seg_channel] = self.max_seg_label_levels[
                seg_channel
            ][0]
        except AttributeError:
            self.max_seg_label_levels = dict()
            self._determine_max_seg_label_levels(seg_channel)

    def _create_df_from_arr(self, arr):
        return (
            xr.DataArray(
                np.moveaxis(arr, 1, 0),
                coords=(
                    self.image_data["segmentations"].coords["seg_channels"][1:],
                    self.image_data["segmentations"].coords["img_name"],
                    np.arange(self.max_seg_label_levels['C0']),
                ),
                dims=("colocalisation_ch", "img_name", "C0_labels",),
            )
            .to_dataframe("is_in_label")
            .reset_index()
            .dropna()
        )

    def measure_overlap(self):
        self._determine_max_seg_label_levels("C0")
        arr = (
            self.image_data["segmentations"]
            .data.map_blocks(
                calculate_overlap,
                drop_axis=[0],
                dtype=np.float64,
                num_of_segs=self.image_data["segmentations"].shape[0],
                preallocate_value=self.max_seg_label_levels["C0"],
            )
            .compute()
        )

        df = self._create_df_from_arr(arr)
        df["is_in_label"] = df["is_in_label"].astype(np.uint16)
        self.results_overlaps = df[
            ["img_name", "C0_labels", "colocalisation_ch", "is_in_label"]
        ]

    def clones_to_keep_as_dict(self, query_for_pd: str):
        return (
            self.results_measurements.query(query_for_pd)
            .groupby("intensity_img_name")
            .agg({"label": lambda x: list(x)})["label"]
            .to_dict()
        )

    def add_clones_and_neighbouring_labels(
        self,
        tot_seg_ch: str = "C0",
        query_for_pd: str = 'intensity_img_channel == "C1" & mean_intensity > 1000',
    ):
        clone_coords, clone_dims = update_1st_coord_and_dim_of_xarr(
            self.segmentations,
            new_coord=[
                f"{tot_seg_ch}",
                f"{tot_seg_ch}_extended",
                f"{tot_seg_ch}_inside_clones",
                f"{tot_seg_ch}_outside_clones",
                f"merged_clones",
                f"{tot_seg_ch}_neighbour_counts",
                f"{tot_seg_ch}_inside_clones_neighbour_counts",
                f"{tot_seg_ch}_outside_clones_neighbour_counts",
            ],
            new_dim="extended_labels_neighbour_counts",
        )

        clones_to_keep = self.clones_to_keep_as_dict(query_for_pd)

        new_label_imgs = get_all_labeled_clones_unmerged_and_merged(
            self.segmentations.loc[tot_seg_ch], clones_to_keep
        )

        self.clones_and_neighbouring_labels = xr.DataArray(
            data=new_label_imgs, coords=clone_coords, dims=clone_dims
        )

## CloneCounter subclasses

In [ ]:
# export
class LazyCloneCounter(CloneCounter):
    def __init__(self, exp_name: str, img_name_regex: str, pixel_size: float):
        super().__init__(exp_name, img_name_regex, pixel_size)

    def add_images(self, **channel_path_globs):
        self.image_data = xr.Dataset(
            {"images": super().add_images(**channel_path_globs)}
        )

    def add_segmentations(self, **channel_path_globs):
        self.image_data["segmentations"] = super().add_segmentations(
            **channel_path_globs
        )

In [ ]:
# export
class PersistentCloneCounter(CloneCounter):
    def __init__(self, exp_name: str, img_name_regex: str, pixel_size: float):
        super().__init__(exp_name, img_name_regex, pixel_size)

    def add_images(self, **channel_path_globs):
        self.image_data = xr.Dataset(
            {"images": super().add_images(**channel_path_globs)}
        ).persist()

    def add_segmentations(self, **channel_path_globs):
        self.image_data["segmentations"] = (
            super().add_segmentations(**channel_path_globs).persist()
        )

## Example using LazyCloneCounter with measure_overlap

In [ ]:
# hide
from dask.distributed import Client

c = Client()
c

/Users/ottomorris/miniconda3/envs/py_clone_det/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 53320 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:53320/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:53320/status,Workers: 4
Total threads: 8,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:53321,Workers: 4
Dashboard: http://127.0.0.1:53320/status,Total threads: 8
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:53331,Total threads: 2
Dashboard: http://127.0.0.1:53333/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:53326,


In [ ]:
foo = LazyCloneCounter("Marcm2a_E7F1", r"a\dg\d\dp\d", 0.275)

foo.add_images(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_imgs/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_imgs/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_imgs/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_imgs/*.tif*",
)

foo.add_segmentations(
    C0="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C0/C0_label_imgs_combined_C3/*.tif*",
    C1="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C1/C1_binaries/*.tif*",
    C2="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C2/C2_label_imgs_v2/*.tif*",
    C3="../current_imaging_analysis/MARCM2A_E7F1_refactoring/C3/C3_label_imgs/*.tif*",
)
foo.make_measurements(extra_properties=["convex_area"],)
foo.measure_overlap()
foo.combine_C0_overlaps_and_measurements()

C1   C2  C3 seg_channel intensity_img_channel  \
img_name C0_labels                                                  
a1g01p1  1           0    0   0          C0                    C0   
         2           0    0   0          C0                    C0   
         3           0    0   0          C0                    C0   
         4           0    3   0          C0                    C0   
         5           0    0   0          C0                    C0   
...                 ..  ...  ..         ...                   ...   
a2g13p3  249         0  150   0          C0                    C0   
         250         0  146   0          C0                    C0   
         251         0  148   0          C0                    C0   
         252         0    0   0          C0                    C0   
         253         0    0   0          C0                    C0   

                   intensity_img_name  area  mean_intensity  centroid-0  \
img_name C0_labels                                                        
a1g01p1  1                    a1g01p1   353     1143.484419    5.025496   
         2                    a1g01p1   613     2521.355628    8.014682   
         3                    a1g01p1   275     2570.625455    5.941818   
         4                    a1g01p1   700     2873.265714   14.550000   
         5                    a1g01p1    33     2838.606061    0.848485   
...                               ...   ...             ...         ...   
a2g13p3  249                  a2g13p3   712     1801.213483  792.443820   
         250                  a2g13p3   308     1980.500000  793.714286   
         251                  a2g13p3   678     2612.631268  799.069322   
         252                  a2g13p3   672     1393.748512  798.055060   
         253                  a2g13p3   212     2926.056604  805.608491   

                    centroid-1  convex_area   area_um2  convex_area_um2  
img_name C0_labels                                                       
a1g01p1  1           97.269122          366  26.695625        27.678750  
         2          222.654160          640  46.358125        48.400000  
         3          383.338182          288  20.796875        21.780000  
         4          519.455714          724  52.937500        54.752500  
         5          579.636364           35   2.495625         2.646875  
...                        ...          ...        ...              ...  
a2g13p3  249        551.751404          739  53.845000        55.886875  
         250        459.652597          318  23.292500        24.048750  
         251        500.057522          696  51.273750        52.635000  
         252        786.084821          689  50.820000        52.105625  
         253        636.231132          219  16.032500        16.561875  

[21336 rows x 13 columns]